In [1]:
# Carga de librerías
import os
import json                 # manejo de archivos JSON
from pathlib import Path    # manejo de rutas
from tqdm.auto import tqdm     # barras de progreso

# LangChain es para construir aplicaciones con modelos de lenguaje
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

print("Librerías importadas correctamente.")

c:\Users\sebax\OneDrive\Desktop\Universidad\2° 2025\Tratamiento Automatico del lenguaje\Proyecto Grande\TADL\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Librerías importadas correctamente.


In [2]:
# --- 1. CONFIGURACIÓN Y LIMPIEZA ---
import shutil

persist_local = "Datos/vector_db_chile_2025"
file_path = Path('Datos/noticias_enriquecidas_2025.json')
batch_size = 1000

# Borrar la base de datos anterior si existe (Clean Slate)
if os.path.exists(persist_local):
    print(f"🗑️ Eliminando base de datos antigua en: {persist_local}...")
    shutil.rmtree(persist_local)
else:
    print("✨ No se encontró base antigua, iniciando desde cero.")

✨ No se encontró base antigua, iniciando desde cero.


In [3]:
# Carga del archivo JSON con las noticias enriquecidas
file_path = Path('Datos/noticias_enriquecidas_2025.json')
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        noticias = json.load(file)
        print('Archivo cargado correctamente.')
        print(f'Número de noticias cargadas: {len(noticias)}')
except FileNotFoundError:
    print(f'Error: El archivo {file_path} no se encuentra.')

Archivo cargado correctamente.
Número de noticias cargadas: 10287


In [4]:
# Creación de chunks a partir de las noticias
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# Configuración inicial
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = []

print("Creando chunks...")
for i, n in enumerate(noticias):
    text = n.get("text") or n.get("contenido") or ""
    if not text:
        continue
    
    # Agregar regiones como metadato si existen
    lista_reg = n.get("regiones", [])
    if isinstance(lista_reg, list):
        regiones_str = ", ".join(lista_reg)
    else:
        regiones_str = str(lista_reg) if lista_reg else ""
    chunks = splitter.split_text(text)
    
    
    for j, chunk in enumerate(chunks):
        metadata = {
            "titulo": n.get("title", "Sin título"),
            "medio": n.get("media_outlet", "Desconocido"),
            "fecha": n.get("fecha", "Sin fecha"),
            "url": n.get("url", "#"),
            "regiones": regiones_str
        }
        docs.append(Document(page_content=chunk, metadata=metadata))
    if (i + 1) % 3000 == 0:
        print(f"→ {i+1} noticias procesadas...")

print(f"Chunks creados: {len(docs)} (a partir de {len(noticias)} noticias)")

# agregar regiones como metadato


Creando chunks...
→ 3000 noticias procesadas...
→ 6000 noticias procesadas...
→ 9000 noticias procesadas...
Chunks creados: 35438 (a partir de 10287 noticias)


In [5]:
# Configuración fija inicial
batch_size = 1000
persist_local = "Datos/vector_db_chile_2025"
os.makedirs(persist_local, exist_ok=True) # Crear directorio si no existe

# Inicializar la base de datos 
db = Chroma(
    embedding_function=embeddings,  
    persist_directory=persist_local,
    collection_name="noticias_chile_2025"
)

print(f"Creando base vectorial con batch de {batch_size}")

# Procesamiento en lotes
total_docs = len(docs)
for i in tqdm(range(0, total_docs, batch_size), desc="Procesando lotes"):
    batch = docs[i : i + batch_size]
    db.add_documents(batch)

# Persistir la base de datos en disco
print(f"\n✅ Base vectorial creada exitosamente en: {persist_local}")




Creando base vectorial con batch de 1000


Procesando lotes: 100%|██████████| 36/36 [10:49<00:00, 18.04s/it]


✅ Base vectorial creada exitosamente en: Datos/vector_db_chile_2025
